In [ ]:
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from skimage.morphology import skeletonize
import os

# ==================== 可调参数 ====================
# 二值化参数
BINARY_THRESHOLD = 100  # 二值化阈值 (60-150, 越低保留细节越多)

# 厚度检测参数
THICKNESS_THRESHOLD = 3  # 厚度检测阈值 (2-5, 越小检测越敏感)
MIN_AREA = 100  # 最小面积过滤 (50-200, 过滤小噪声)

# 轮廓生成参数
OUTLINE_WIDTH = 2  # 轮廓线宽 (1-3, 生成的边框厚度)

# ==================== 核心函数 ====================

def convert_fills_to_outlines(binary_img):
    """将粗区域转换为边框线条（基于厚度检测）"""

    # 1. 识别连通组件
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary_img)
    result_img = binary_img.copy()
    processed_count = 0

    for i in range(1, num_labels):  # 跳过背景标签0
        area = stats[i, cv2.CC_STAT_AREA]

        # 面积过滤
        if area < MIN_AREA:
            continue

        # 提取该组件
        component_mask = (labels == i).astype(np.uint8) * 255

        # 厚度检测
        if is_thick_region(component_mask, THICKNESS_THRESHOLD):
            # 生成边框
            outline_mask = create_outline_from_fill(component_mask)

            # 替换：移除原填充，添加边框
            result_img[labels == i] = 0  # 扣空操作
            result_img = cv2.bitwise_or(result_img, outline_mask)  # 添加边框
            processed_count += 1

    print(f"Processed {processed_count} thick regions")
    return result_img

def is_thick_region(component_mask, thickness_threshold):
    """使用形态学腐蚀检测区域厚度"""

    # 创建圆形kernel
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (thickness_threshold, thickness_threshold))

    # 腐蚀：只有厚度>=threshold的区域能存活
    eroded = cv2.erode(component_mask, kernel)

    # 如果腐蚀后还有残留，说明是粗区域
    return np.sum(eroded) > 0

def binarize_image(img):
    """二值化图像"""
    _, binary = cv2.threshold(img, BINARY_THRESHOLD, 255, cv2.THRESH_BINARY_INV)
    return binary

def create_outline_from_fill(fill_mask):
    """从填充区域生成边框"""

    # 查找轮廓
    contours, _ = cv2.findContours(fill_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    outline_mask = np.zeros_like(fill_mask)

    for contour in contours:
        # 绘制轮廓边框
        cv2.drawContours(outline_mask, [contour], -1, 255, thickness=OUTLINE_WIDTH)

    return outline_mask

def process_vectorization_input():
    """处理矢量化输入图像：填充区域转边框"""

    print(f"Loading image: {VECTORIZATION_INPUT}")

    # 检查文件是否存在
    if not os.path.exists(VECTORIZATION_INPUT):
        print(f"ERROR: Input file not found: {VECTORIZATION_INPUT}")
        return None

    # 加载图像
    img = cv2.imread(VECTORIZATION_INPUT, cv2.IMREAD_GRAYSCALE)
    print(f"Original image shape: {img.shape}")

    # 二值化
    binary = binarize_image(img)
    print(f"Binary threshold: {BINARY_THRESHOLD}")

    # 执行扣空处理
    outline_img = convert_fills_to_outlines(binary)

    # 保存结果
    os.makedirs(INTERMEDIATE_FILES_PATH, exist_ok=True)

    binary_path = os.path.join(INTERMEDIATE_FILES_PATH, f"{TARGET_NAME}_binary.png")
    outline_path = os.path.join(INTERMEDIATE_FILES_PATH, f"{TARGET_NAME}_outline.png")

    cv2.imwrite(binary_path, binary)
    cv2.imwrite(outline_path, outline_img)

    print(f"Binary image saved: {binary_path}")
    print(f"Outline image saved: {outline_path}")

    return outline_img

def visualize_processing_steps():
    """可视化处理步骤"""

    # 加载原图
    original = cv2.imread(VECTORIZATION_INPUT, cv2.IMREAD_GRAYSCALE)

    # 加载处理结果
    binary_path = os.path.join(INTERMEDIATE_FILES_PATH, f"{TARGET_NAME}_binary.png")
    outline_path = os.path.join(INTERMEDIATE_FILES_PATH, f"{TARGET_NAME}_outline.png")

    if os.path.exists(binary_path) and os.path.exists(outline_path):
        binary = cv2.imread(binary_path, cv2.IMREAD_GRAYSCALE)
        outline = cv2.imread(outline_path, cv2.IMREAD_GRAYSCALE)

        # 创建对比图
        fig, axes = plt.subplots(1, 3, figsize=(15, 5))

        axes[0].imshow(original, cmap='gray')
        axes[0].set_title('Original (Qwen Edit)')
        axes[0].axis('off')

        axes[1].imshow(binary, cmap='gray')
        axes[1].set_title('Binary (Inverted)')
        axes[1].axis('off')

        axes[2].imshow(outline, cmap='gray')
        axes[2].set_title('Fill→Outline (Processed)')
        axes[2].axis('off')

        plt.tight_layout()
        plt.show()
    else:
        print("Processed images not found. Please run process_vectorization_input() first.")

# ==================== 执行处理 ====================
print("=== Fill-to-Outline Processing ===")
outline_result = process_vectorization_input()

if outline_result is not None:
    print("✅ Processing completed successfully!")
    # 显示对比
    visualize_processing_steps()
else:
    print("❌ Processing failed!")


In [ ]:
# ==================== 骨架化处理 ====================
# 基于上一步的outline图片进行骨架化处理

import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from skimage.morphology import skeletonize
import os

# 骨架化参数
SKELETON_MIN_LENGTH = 3  # 最小骨架长度 (降低到3, 保留更多内容)
ENDPOINT_CONNECTION_DISTANCE = 8  # 端点连接距离 (增加到8, 连接更多断点)

def extract_skeleton(outline_img):
    """提取骨架：将多像素线条转换为单像素骨架"""

    print(f"输入图像形状: {outline_img.shape}")

    # 1. 基础骨架化
    print("执行骨架化...")
    skeleton = skeletonize(outline_img > 0)
    print(f"骨架化完成，骨架像素数: {np.sum(skeleton)}")

    # 2. 距离变换（用于线宽估算）
    print("计算距离变换...")
    distance_transform = cv2.distanceTransform(outline_img, cv2.DIST_L2, 5)
    print(f"距离变换完成，最大距离: {distance_transform.max():.2f}")

    # 3. 骨架优化
    print("优化骨架质量...")
    optimized_skeleton = optimize_skeleton(skeleton)
    print(f"优化完成，最终骨架像素数: {np.sum(optimized_skeleton)}")

    return optimized_skeleton, distance_transform

def optimize_skeleton(skeleton):
    """优化骨架质量"""

    # 1. 连接近邻断点
    print("  连接断开的端点...")
    skeleton_connected = connect_nearby_endpoints(skeleton)

    # 2. 移除毛刺
    print("  移除短毛刺...")
    skeleton_cleaned = remove_spurs(skeleton_connected)

    # 3. 平滑骨架
    print("  平滑骨架线条...")
    skeleton_smoothed = smooth_skeleton(skeleton_cleaned)

    return skeleton_smoothed

def connect_nearby_endpoints(skeleton):
    """连接距离较近的端点"""

    # 查找端点
    endpoints = find_skeleton_endpoints(skeleton)
    print(f"    找到 {len(endpoints)} 个端点")

    # 转换为uint8类型用于cv2.line
    result = skeleton.astype(np.uint8)
    connections_made = 0

    for i, ep1 in enumerate(endpoints):
        for j, ep2 in enumerate(endpoints[i+1:], i+1):
            distance = np.linalg.norm(np.array(ep1) - np.array(ep2))

            if distance <= ENDPOINT_CONNECTION_DISTANCE:
                # 连接两个端点
                cv2.line(result, ep1[::-1], ep2[::-1], 1, 1)
                connections_made += 1

    print(f"    连接了 {connections_made} 对端点")
    return result.astype(bool)

def find_skeleton_endpoints(skeleton):
    """查找骨架端点"""

    # 使用3x3卷积核计算邻域
    kernel = np.array([[1, 1, 1],
                       [1, 10, 1],
                       [1, 1, 1]], dtype=np.uint8)

    neighbor_count = cv2.filter2D(skeleton.astype(np.uint8), -1, kernel)

    # 端点：自己是1，邻域和为11（10+1）
    endpoints = np.where((skeleton == 1) & (neighbor_count == 11))

    return list(zip(endpoints[0], endpoints[1]))

def remove_spurs(skeleton, min_length=None):
    """移除毛刺（短分支）"""

    if min_length is None:
        min_length = SKELETON_MIN_LENGTH

    # 查找连通组件
    num_labels, labels = cv2.connectedComponents(skeleton.astype(np.uint8))
    result = np.zeros_like(skeleton)
    removed_count = 0

    for i in range(1, num_labels):
        component = (labels == i)
        component_size = np.sum(component)

        # 只保留足够长的组件
        if component_size >= min_length:
            result[component] = 1
        else:
            removed_count += 1

    print(f"    移除了 {removed_count} 个短毛刺")
    return result

def smooth_skeleton(skeleton):
    """平滑骨架线条"""

    # 问题：MORPH_OPEN 过于激进，会删除太多内容
    # 解决方案：使用更温和的平滑方法，或者跳过平滑步骤

    # 方法1：跳过平滑（推荐）
    return skeleton

    # 方法2：使用更小的kernel（如果确实需要平滑）
    # kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (1, 1))
    # smoothed = cv2.morphologyEx(skeleton.astype(np.uint8), cv2.MORPH_OPEN, kernel)
    # return smoothed.astype(bool)

def process_skeletonization():
    """处理骨架化：直接从文件读取outline图片生成骨架"""

    # 直接从文件加载outline图片
    outline_path = os.path.join(INTERMEDIATE_FILES_PATH, f"{TARGET_NAME}_outline.png")

    if not os.path.exists(outline_path):
        print(f"ERROR: Outline file not found: {outline_path}")
        print("请先运行上一步的fill-to-outline处理")
        return None, None

    print(f"从文件加载outline图片: {outline_path}")
    outline_img = cv2.imread(outline_path, cv2.IMREAD_GRAYSCALE)

    if outline_img is None:
        print(f"ERROR: 无法读取图片文件: {outline_path}")
        return None, None

    print(f"Outline图片形状: {outline_img.shape}")
    print(f"Outline图片像素范围: {outline_img.min()} - {outline_img.max()}")

    # 执行骨架化
    skeleton, distance_transform = extract_skeleton(outline_img)

    # 保存结果
    skeleton_path = os.path.join(INTERMEDIATE_FILES_PATH, f"{TARGET_NAME}_skeleton.png")
    distance_path = os.path.join(INTERMEDIATE_FILES_PATH, f"{TARGET_NAME}_distance.png")

    # 保存骨架（转换为0-255范围）
    skeleton_uint8 = (skeleton * 255).astype(np.uint8)
    cv2.imwrite(skeleton_path, skeleton_uint8)

    # 保存距离变换（归一化到0-255）
    distance_normalized = cv2.normalize(distance_transform, None, 0, 255, cv2.NORM_MINMAX)
    cv2.imwrite(distance_path, distance_normalized.astype(np.uint8))

    print(f"骨架图片保存: {skeleton_path}")
    print(f"距离变换保存: {distance_path}")

    return skeleton, distance_transform

def debug_skeletonization_steps(outline_img):
    """调试骨架化各个步骤，显示中间结果"""

    print("=== 骨架化步骤调试 ===")

    # 步骤1：基础骨架化
    print("步骤1：基础骨架化")
    skeleton_raw = skeletonize(outline_img > 0)
    skeleton_pixels = np.sum(skeleton_raw)
    print(f"  原始骨架像素数: {skeleton_pixels}")

    # 步骤2：端点检测
    print("步骤2：端点检测")
    endpoints = find_skeleton_endpoints(skeleton_raw)
    print(f"  检测到端点数量: {len(endpoints)}")

    # 步骤3：连接端点
    print("步骤3：连接端点")
    skeleton_connected = connect_nearby_endpoints(skeleton_raw)
    connected_pixels = np.sum(skeleton_connected)
    print(f"  连接后像素数: {connected_pixels}")

    # 步骤4：移除毛刺
    print("步骤4：移除毛刺")
    skeleton_cleaned = remove_spurs(skeleton_connected)
    cleaned_pixels = np.sum(skeleton_cleaned)
    print(f"  清理后像素数: {cleaned_pixels}")

    # 步骤5：平滑处理
    print("步骤5：平滑处理")
    skeleton_smoothed = smooth_skeleton(skeleton_cleaned)
    final_pixels = np.sum(skeleton_smoothed)
    print(f"  最终像素数: {final_pixels}")

    # 距离变换
    distance_transform = cv2.distanceTransform(outline_img, cv2.DIST_L2, 5)

    return {
        'original': outline_img,
        'skeleton_raw': skeleton_raw,
        'skeleton_connected': skeleton_connected,
        'skeleton_cleaned': skeleton_cleaned,
        'skeleton_final': skeleton_smoothed,
        'distance_transform': distance_transform,
        'stats': {
            'raw_pixels': skeleton_pixels,
            'connected_pixels': connected_pixels,
            'cleaned_pixels': cleaned_pixels,
            'final_pixels': final_pixels,
            'endpoints': len(endpoints)
        }
    }

def visualize_skeletonization():
    """可视化骨架化结果：从文件读取所有图片"""

    # 从文件加载原图
    original = cv2.imread(VECTORIZATION_INPUT, cv2.IMREAD_GRAYSCALE)
    if original is None:
        print(f"ERROR: 无法读取原图: {VECTORIZATION_INPUT}")
        return

    # 从文件加载outline图片
    outline_path = os.path.join(INTERMEDIATE_FILES_PATH, f"{TARGET_NAME}_outline.png")
    outline = cv2.imread(outline_path, cv2.IMREAD_GRAYSCALE)
    if outline is None:
        print(f"ERROR: 无法读取outline图片: {outline_path}")
        return

    # 从文件加载骨架图片
    skeleton_path = os.path.join(INTERMEDIATE_FILES_PATH, f"{TARGET_NAME}_skeleton.png")
    skeleton = cv2.imread(skeleton_path, cv2.IMREAD_GRAYSCALE)
    if skeleton is None:
        print(f"ERROR: 无法读取骨架图片: {skeleton_path}")
        return

    print("所有图片加载成功，显示对比图...")

    # 创建对比图
    fig, axes = plt.subplots(1, 3, figsize=(15, 5))

    axes[0].imshow(original, cmap='gray')
    axes[0].set_title('Original (Qwen Edit)')
    axes[0].axis('off')

    axes[1].imshow(outline, cmap='gray')
    axes[1].set_title('Outline (Multi-pixel)')
    axes[1].axis('off')

    axes[2].imshow(skeleton, cmap='gray')
    axes[2].set_title('Skeleton (Single-pixel)')
    axes[2].axis('off')

    plt.tight_layout()
    plt.show()

def visualize_skeletonization_steps(outline_img):
    """可视化骨架化各个步骤的详细对比"""

    # 执行调试步骤
    debug_results = debug_skeletonization_steps(outline_img)

    # 创建6个子图的对比
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))

    # 第一行：原始处理步骤
    axes[0,0].imshow(debug_results['original'], cmap='gray')
    axes[0,0].set_title('1. Original Outline')
    axes[0,0].axis('off')

    axes[0,1].imshow(debug_results['skeleton_raw'], cmap='gray')
    axes[0,1].set_title(f'2. Raw Skeleton\n({debug_results["stats"]["raw_pixels"]} pixels)')
    axes[0,1].axis('off')

    axes[0,2].imshow(debug_results['skeleton_connected'], cmap='gray')
    axes[0,2].set_title(f'3. Connected\n({debug_results["stats"]["connected_pixels"]} pixels)')
    axes[0,2].axis('off')

    # 第二行：优化步骤
    axes[1,0].imshow(debug_results['skeleton_cleaned'], cmap='gray')
    axes[1,0].set_title(f'4. Cleaned\n({debug_results["stats"]["cleaned_pixels"]} pixels)')
    axes[1,0].axis('off')

    axes[1,1].imshow(debug_results['skeleton_final'], cmap='gray')
    axes[1,1].set_title(f'5. Final Skeleton\n({debug_results["stats"]["final_pixels"]} pixels)')
    axes[1,1].axis('off')

    # 距离变换
    distance_normalized = cv2.normalize(debug_results['distance_transform'], None, 0, 255, cv2.NORM_MINMAX)
    axes[1,2].imshow(distance_normalized, cmap='hot')
    axes[1,2].set_title('6. Distance Transform')
    axes[1,2].axis('off')

    plt.tight_layout()
    plt.show()

    # 打印统计信息
    stats = debug_results['stats']
    print(f"\n=== 骨架化统计信息 ===")
    print(f"原始骨架像素: {stats['raw_pixels']}")
    print(f"连接后像素: {stats['connected_pixels']}")
    print(f"清理后像素: {stats['cleaned_pixels']}")
    print(f"最终像素: {stats['final_pixels']}")
    print(f"检测到端点: {stats['endpoints']}")

    # 分析问题
    if stats['final_pixels'] < stats['raw_pixels'] * 0.1:
        print("⚠️ 警告：最终骨架像素数过少，可能存在问题")
    if stats['endpoints'] == 0:
        print("⚠️ 警告：未检测到端点，骨架可能不完整")

    return debug_results

# 执行骨架化处理
print("=== Skeletonization Processing ===")
skeleton_result, distance_result = process_skeletonization()

if skeleton_result is not None:
    print("✅ 骨架化处理完成!")

    # 加载outline图片进行详细步骤分析
    outline_path = os.path.join(INTERMEDIATE_FILES_PATH, f"{TARGET_NAME}_outline.png")
    outline_img = cv2.imread(outline_path, cv2.IMREAD_GRAYSCALE)

    if outline_img is not None:
        print("\n=== 详细步骤分析 ===")
        # 显示详细的中间步骤
        debug_results = visualize_skeletonization_steps(outline_img)

        # 显示最终对比
        print("\n=== 最终结果对比 ===")
        visualize_skeletonization()
    else:
        print("❌ 无法加载outline图片进行详细分析")
else:
    print("❌ 骨架化处理失败!")
